## __Data Gathering__

**Goal:** This notebook gathers the data from BoardGameGeek.com (BGG) and Miniature Market (MM; a game retailer).  In order to limit the amount of data gathered from BGG - which has more games than in its database than any retailer has on hand - I start by gathering game data from Miniature Market, and then proceed to gather the data from BGG for all games in the MM dataset.

In [2]:
#For Boardgamegeek.com
# !pip install boardgamegeek
from boardgamegeek import BoardGameGeek as BGG
import boardgamegeek
import time

#For MiniatureMarket.com
import requests
# from tqdm import tqdm

#Developing data
import json
import pandas as pd

## __Get prices for games from Miniature Market__

Gather all the game data from Miniature Market's website.

API docs [here]('http://unbxd.com/documentation/site-search/v2-search-rest-api/')

In [66]:
def get_data_from_page(game_json):
    temp_data_dict = {}
    for i in game_json['response']['products']:
        temp_data_dict[i['sku']] = {'title': i.get('title'),
                                    'availability': i.get('availability'), 
                                    'bggeek': i.get('bggeek'),
                                    'category': i.get('category'),
                                    'description': i.get('description'),
                                    'entity_id': i.get('entity_id'), 
                                    'genre': [g for g in i.get('genre')],
                                    'google_shopping_name': i['google_shopping_name'],
                                    'manufacturer': i['manufacturer'],
                                    'normal_price': i.get('normal_price'),
                                    'min_players': i.get('player_min'),
                                    'max_players': i.get('player_max'), 
                                    'play_time': i.get('playing_time'), 
                                    'mm_price': i.get('price'),
                                    'mm_url': i.get('productUrl'),
                                    'product_tag': [g for g in i.get('product_tag')],
                                    'quantity': i.get('quantity'),
                                    'rating_count': i.get('rating_count'), 
                                    'rating_percentage': i.get('rating_percentage'),
                                    'rating_star': i.get('rating_star'),
                                    'recent_sales': i.get('recent_sales'),
                                    'regular_price': i.get('regular_price'),
                                    'sub_genres': [g for g in i.get('sub_genre_1')],
                                    'weight': i.get('weight'),
                                    'score': i.get('score')}
    return temp_data_dict

In [79]:
#Base URL for data
games_per_page = 32
start = 0
url = 'https://search.unbxd.io/fb500edbf5c28edfa74cc90561fe33c3/prod-miniaturemarket-com811741582229555/category?format=json&version=V2&start={}&rows={}&pagetype=boolean&p=categoryPath%3A%22Board+Games%22&filter=categoryPath1_fq:%22Board%20Games%22'.format(str(start), str(games_per_page))

#Get total number of games
result = requests.get(url)
game_count = result.json()['response']['numberOfProducts']
print('Total games at Miniature Market: ', game_count)

def get_mm_data(games_per_page=32, game_count=game_count):
    all_games_data = {}
    #Cycle through all pages
    for start in range(0, game_count+games_per_page, games_per_page):
        url = 'https://search.unbxd.io/fb500edbf5c28edfa74cc90561fe33c3/prod-miniaturemarket-com811741582229555/category?format=json&version=V2&start={}&rows={}&pagetype=boolean&p=categoryPath%3A%22Board+Games%22&filter=categoryPath1_fq:%22Board%20Games%22'.format(str(start), str(games_per_page))
        result = requests.get(url)
        game_json = result.json()
        
        #Add data from MiniatureMarket JSON to dictionary
        temp_data = get_data_from_page(game_json)

        #Update dict with all game data
        all_games_data.update(temp_data)
    time.sleep(3)

    return all_games_data

Total games at Miniature Market:  5791


In [80]:
all_games_data = get_mm_data(games_per_page=32, game_count=32)

In [ ]:
#Print info on data
print('Number of games in dataset: ', len(all_games_data), '\n')
print('Keys for each game:', '\n', '------------------')

def list_keys(dataset, dummy_key):
    all_keys = [key for key in dataset[dummy_key].keys()] #WOCC75670 is a "random" ID chosen
    for k in all_keys:
        print(k)

list_keys(all_games_data, 'WOCC75670')

#Data examples
# all_games_data['SJG1577']
# all_games_data['WOCC75670']

In [83]:
import json
#Save MM data to file
with open('mm_data.json', 'w') as fp:
    json.dump(all_games_data, fp)

In [2]:
import json
#Read in JSON file
with open('mm_data.json') as json_file:
    all_games_data = json.load(json_file)

In [ ]:
#Update MM data
def update_data():
    

In [5]:
#Get all IDs from MM data
dict_of_names_ids = {}
for game in all_games_data.values():
#     print(game['bggeek'], type(game['bggeek']))
    if (game['bggeek'] != 0) or (game['bggeek'] != ' '): 
        if (game['bggeek'] != 277896):
            try: 
                game['bggeek'] = int(game['bggeek'])
                dict_of_names_ids[game['bggeek']] = game['google_shopping_name']
            except:
                continue

print(len(dict_of_names_ids))

4745


## __Getting ratings for a list of games__
The following code takes the boardgamegeek ID codes from the Miniature Market Data above, and gathers associated BGG data.

The [`boardgamegeek` package]('https://lcosmin.github.io/boardgamegeek/') is used to easily draw data from BGG's XML database.

In [329]:
#Alternative method - but only works with names (which may not be identical across e-commerce platforms)
bgg = BGG()
# for name in all_mm_names[:10]:
x = bgg.search('agricola', search_type=['boardgame', 'boardgameexpansion'], exact=False)
print(x)
for i in x:
    print(bgg.game(game_id=i.id).name)

#Example of names not being identical
# bgg.game(game_id=272638).name

# 'Root: The Exiles and Partisans Deck'
# 'Root: Exiles & Partisans Deck'

[Thing (id: 31260), Thing (id: 197633), Thing (id: 200680), Thing (id: 59158), Thing (id: 9364), Thing (id: 113413), Thing (id: 6901), Thing (id: 45020), Thing (id: 216651), Thing (id: 216650), Thing (id: 216652), Thing (id: 216654), Thing (id: 216649), Thing (id: 216655), Thing (id: 7688), Thing (id: 11670), Thing (id: 10799), Thing (id: 38733), Thing (id: 37235), Thing (id: 206805), Thing (id: 292850), Thing (id: 119890), Thing (id: 145355), Thing (id: 130752), Thing (id: 255262), Thing (id: 242392), Thing (id: 234218), Thing (id: 131403), Thing (id: 150732), Thing (id: 111165), Thing (id: 132592), Thing (id: 247720), Thing (id: 263965), Thing (id: 173558), Thing (id: 242391), Thing (id: 271144), Thing (id: 109012), Thing (id: 311545), Thing (id: 210625), Thing (id: 205418), Thing (id: 43018), Thing (id: 257344), Thing (id: 242390), Thing (id: 166463), Thing (id: 85704), Thing (id: 183316), Thing (id: 242388), Thing (id: 39090), Thing (id: 103182), Thing (id: 242389), Thing (id: 1308

In [30]:
from boardgamegeek import BoardGameGeekError

#Now ingesting game IDs from MM data
def get_bgg_data(dict_of_names_ids):
    #Instantiate the class
    bgg = BGG()
    
    #Begin retrieving data
    bgg_data = {}
    for game_id, game_name in dict_of_names_ids.items():
        print(game_name, '- ', game_id)
        try:
            #find game by ID or name
            game = bgg.game(name=game_name, game_id=game_id)
            game_data = {'name' : game.name,
                         'rating_average' : game.rating_average,
                         'year' : game.year,
                         'artists' : [a for a in game.artists],
                         'categories' : [c for c in game.categories],
                             'designers' : [d for d in game.designers],
                             'is_expansion' : game.expansion,
                             'expands' : [e.id for e in game.expands],
                             'expansion_ids' : [i.id for i in game.expansions],
                             'families' : [f for f in game.families],
                             'implementations' : [i for i in game.implementations],
                             'min_players' : game.min_players,
                             'max_players' : game.max_players,
                             'mechanics' : [m for m in game.mechanics],
                             'min_age' : game.min_age,
                             'playing_time' : game.playing_time,
                             'publishers' : [p for p in game.publishers],
                             'ranks' : game.ranks,
                             'rating_average' : game.rating_average,
                             'rating_average_weight' : game.rating_average_weight,
                             'rating_bayes_average' : game.rating_bayes_average,
                              'rating_median' : game.rating_median,
                             'rating_num_weights' : game.rating_num_weights,
                             'rating_stddev' : game.rating_stddev,
                             'users_commented' : game.users_commented,
                             'users_owned' : game.users_owned,
                             'users_rated' : game.users_rated,
                             'users_trading' : game.users_trading,
                             'users_wanting' : game.users_wanting,
                             'users_wishing' : game.users_wishing
                             }  
            #Get all this data     
            bgg_data[game.id] = game_data
        except BoardGameGeekError: 
            pass
    
    return bgg_data

In [26]:
# test_bgg = dict(list(dict_of_names_ids.items())[0: 20])  

In [31]:
t = time.time()
bgg_data = get_bgg_data(dict_of_names_ids)
print('time elapsed: ', time.time() - t)
print('Total number of games with data: ', len(bgg_data))
# bgg_data

Scooby Doo: Betrayal at Mystery Mansion -  302809
Century: Golem Edition - Eastern Mountains -  283619
Undaunted: North Africa -  290359
Root: The Vagabond Pack -  277896
Unmatched: Cobble & Fog -  294484
Fallout Shelter: The Board Game -  293889
Pandemic: Hot Zone - North America -  301919
Root: Exiles & Partisans Deck -  272638
The Lord of the Rings: Journeys in Middle-earth - Shadowed Paths -  299597
Napoleon Returns 1815 -  276337
Imperial Struggle -  206480
El Maestro -  264843
WWE: Headlock, Paper, Scissors -  298190
The Great Cake Escape -  300401
Philadelphia 1777 -  290681
Donning the Purple -  221155
Dark Souls: Executioner's Chariot -  204004
Illuminati (2nd Edition): Alternative Truths Expansion -  298175
Deadly Doodles 2 Expansion -  288816
Deep Vents -  303159
Maximum Apocalypse: Legendary Edition -  275564
Maximum Apocalypse: Jurassic Perils Expansion -  249137
Marvel Villainous: Infinite Power -  302336
Imperial Settlers: Rise of the Empire -  299277
Hannibal & Hamilcar

In [32]:
# from boardgamegeek import things
for v in bgg_data.values():
    for i in v:
        if not isinstance(i, str) or not isinstance(v, dict):
            print(type(i))

In [33]:
#Save MM data to file
with open('bgg_data.json', 'w') as fp:
    json.dump(bgg_data, fp)

In [34]:
#Create pandas dataframe
df = pd.DataFrame.from_dict(bgg_data).T
# df.drop()
df.head()

,name,rating_average,year,artists,categories,designers,is_expansion,expands,expansion_ids,families,...,rating_bayes_average,rating_median,rating_num_weights,rating_stddev,users_commented,users_owned,users_rated,users_trading,users_wanting,users_wishing
302809,Betrayal at Mystery Mansion,7.23633,2020,[],"[Adventure, Exploration, Miniatures]","[Banana Chan, Noah Cohen, Rob Daviau, Brian Neff]",False,[],[],"[Game: Betrayal (Avalon Hill), Theme: Spooky O...",...,5.56953,0,6,1.64706,53,630,79,5,85,403
283619,Century: Golem Edition – Eastern Mountains,7.70797,2019,"[Atha Kanaani, Chris Quilliams]",[Fantasy],[Emerson Matsuuchi],False,[],[],[Game: Century (Plan B Games)],...,5.82307,0,5,1.00562,80,1072,267,5,133,584
290359,Undaunted: North Africa,8.26482,2020,[Roland MacDonald],"[Wargame, World War II]","[Trevor Benjamin, David Thompson (I)]",False,[],[313441],"[Country: Libya, Game: Undaunted Normandy, His...",...,5.88642,0,17,1.24028,96,1657,277,11,169,834
294484,Unmatched: Cobble & Fog,8.74445,2020,[Andrew Thompson],"[Book, Card Game, Fantasy, Fighting, Miniature...","[Rob Daviau, Justin D. Jacobson, Chris Leder]",False,[],[],"[Books: Dracula, Books: Sherlock Holmes, Game:...",...,5.94348,0,4,1.06607,69,1010,278,4,167,682
293889,Fallout Shelter: The Board Game,7.40626,2020,[],[Video Game Theme],[Andrew Fischer],False,[],[],"[Game: Fallout, Theme: Post-Apocalyptic]",...,5.80115,0,10,1.17744,61,887,318,10,80,529


In [37]:
#Print info on data
print('Number of games in dataset: ', len(all_games_data), '\n')
print('Keys for each game:', '\n', '------------------')

list_keys(bgg_data, 302809) #BGG ID (as int, not str)...not MM ID

Number of games in dataset:  5855 

Keys for each game: 
 ------------------
name
rating_average
year
artists
categories
designers
is_expansion
expands
expansion_ids
families
implementations
min_players
max_players
mechanics
min_age
playing_time
publishers
ranks
rating_average_weight
rating_bayes_average
rating_median
rating_num_weights
rating_stddev
users_commented
users_owned
users_rated
users_trading
users_wanting
users_wishing


### **Other data**

Some other data from BGG that may be of interest later (by way of personal reminder).

In [284]:
#Hot items list from BGG
bgg_hot_items = bgg.hot_items('boardgame')
for i in bgg_hot_items:
    print('{}: {} ({}) - {}'.format(i.rank, i.name, i.id, i.yearpublished))

1: Dune: Imperium (316554) - None
2: Massive Darkness 2: Hellscape (315610) - 2021
3: Gloomhaven: Jaws of the Lion (291457) - 2020
4: Pendulum (312804) - 2020
5: Spirit Island (162886) - 2017
6: Gloomhaven (174430) - 2017
7: Marvel Villainous: Infinite Power (302336) - 2020
8: Fort (296912) - 2020
9: Santorini New York (315631) - 2020
10: Brass: Birmingham (224517) - 2018
11: The Crew: The Quest for Planet Nine (284083) - 2019
12: Die Quacksalber von Quedlinburg: Die Alchemisten (316597) - 2020
13: Terraforming Mars (167791) - 2016
14: Twilight Imperium (Fourth Edition) (233078) - 2017
15: Marvel Champions: The Card Game (285774) - 2019
16: Tainted Grail: The Fall of Avalon (264220) - 2019
17: Nusfjord (234277) - 2017
18: Wingspan (266192) - 2019
19: Massive Darkness (197070) - 2017
20: Arkham Horror: The Card Game (205637) - 2016
21: Root (237182) - 2018
22: Descent: Journeys in the Dark (Second Edition) (104162) - 2012
23: Everdell (199792) - 2018
24: Scythe (169786) - 2016
25: The Q